In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# from animal_shelter import AnimalShelter

from aac_crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "newPass123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image (DONE) # IM HERE SO FAR
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
# FIX ME Also remember to include a unique identifier such as your name or date
# html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

# logo goes here (formatted)

logo = html.A(
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image),
        style={'height': '90px'}
    ),
    href="https://www.snhu.edu",
    target="_blank"
)

# Required Specifications from Document

# Rescue Type
rescue_breed_map = {
    'water': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
    'mountain': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
    'disaster': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
}

# Training Age
rescue_constraints = {
    'water': {'sex': 'Intact Female', 'age_min': 26, 'age_max': 156},
    'mountain': {'sex': 'Intact Male', 'age_min': 26, 'age_max': 156},
    'disaster': {'sex': 'Intact Male', 'age_min': 20, 'age_max': 300}
}

app.layout = html.Div([
    #    html.Div(id='hidden-div', style={'display':'none'}),
    logo,
    html.Center(html.B(html.H1('Felix Guzman CS-340 Dashboard'))),
    html.Hr(),
    html.Div(

        # FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        [
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'water'},
                    {'label': 'Mountain Rescue', 'value': 'mountain'},
                    {'label': 'Disaster Rescue', 'value': 'disaster'},
                    {'label': 'Reset', 'value': 'all'}
                ],
                value='all',
                labelStyle={'display': 'inline-block', 'margin-right': '10px'}
            )
        ]

    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         # FIXME: Set up the features for your interactive data table to make it user-friendly for your client
                         # If you completed the Module Six Assignment, you can copy in the code you created here

                         row_selectable="single",
                         selected_rows=[0],
                         sort_action="native",
                         filter_action="native",
                         page_size=10,
                         style_table={'overflowX': 'auto', 'maxHeight': '400px', 'overflowY': 'auto'}

                         ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',

                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])


#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ## FIX ME Add code to filter interactive data table with MongoDB queries
    #
    #
    #        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    #        data=df.to_dict('records')
    #
    #
    #        return (data,columns)
    if filter_type == 'all' or filter_type is None:
        records = db.read({})

    # Water rescue
    elif filter_type == 'water':
        breeds = rescue_breed_map['water']
        cons = rescue_constraints['water']
        records = db.read({
            "animal_type": "Dog",
            "breed": {"$in": breeds},
            "sex_upon_outcome": cons['sex'],
            "age_upon_outcome_in_weeks": {"$gte": cons['age_min'], "$lte": cons['age_max']}
        })
    # Mountain rescue
    elif filter_type == 'mountain':
        breeds = rescue_breed_map['mountain']
        cons = rescue_constraints['mountain']
        records = db.read({
            "animal_type": "Dog",
            "breed": {"$in": breeds},
            "sex_upon_outcome": cons['sex'],
            "age_upon_outcome_in_weeks": {"$gte": cons['age_min'], "$lte": cons['age_max']}
        })
    # Disaster Rescue
    elif filter_type == 'disaster':
        breeds = rescue_breed_map['disaster']
        cons = rescue_constraints['disaster']
        records = db.read({
            "animal_type": "Dog",
            "breed": {"$in": breeds},
            "sex_upon_outcome": cons['sex'],
            "age_upon_outcome_in_weeks": {"$gte": cons['age_min'], "$lte": cons['age_max']}
        })
    else:
        records = db.read({})

    # Return if nothing found to avoid errors
    if not records:
        return []

    dff = pd.DataFrame.from_records(records)
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)
    dff.reset_index(drop=True, inplace=True)
    return dff.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])  # im here
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    # return [
    #    dcc.Graph(
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )
    # ]

    if viewData is None:
        dff = df.copy()
    else:
        dff = pd.DataFrame.from_dict(viewData)

    if dff.empty:
        return []

    # Preparing breed counts to avoid missing value errors
    breed_series = dff['breed'].fillna('Unknown').astype(str)
    vc = breed_series.value_counts()

    # Making a graph that can handle large amounts of data via threshold and other seperators

    # To avoid a long tail of distinct 0.1% breeds I just aggregate it into the "Other" type.
    threshold = 12 # This can be modified.
    if len(vc) > threshold:
        top = vc.head(threshold)
        other_sum = vc.iloc[threshold:].sum()
        top['Other'] = other_sum
        pie_df = top.reset_index()
        pie_df.columns = ['breed', 'count']
        fig = px.pie(pie_df, names='breed', values='count',
                     title='Preferred Animals (top {} + Other)'.format(threshold))
    else:
        # small number of breeds — show full pie
        fig = px.pie(dff, names='breed', title='Preferred Animals')

    # To make the pie as readable as possible we will show percent and label slices appropriately as well as adding a hover for our labels
    fig.update_traces(textinfo='percent+label', hoverinfo='label+percent+value')

    # Layout tweaks so pie isn't tiny (previous issue I had) and legend sits to the right and adding scrolling
    fig.update_layout(
        width=900,
        height=520,
        legend=dict(orientation='v', y=0.5, x=1.02, traceorder='normal'),
        margin=dict(l=20, r=220, t=40, b=20)
    )

    return [
        dcc.Graph(
            figure=fig
        )
    ]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:  # had to add this b/c error due to return type
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None:
        return
    elif index is None:
        return

    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)

Dash app running on http://127.0.0.1:15178/
